This notebook demonstrates how to do co-refinement of several datasets with *refnx*

In [ ]:
%matplotlib inline
from __future__ import print_function, division

import os.path

from refnx.dataset import ReflectDataset
from refnx.analysis import Transform, CurveFitter, Objective, GlobalObjective, Parameter
from refnx.reflect import SLD, ReflectModel

import corner
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# what is the refnx version
# it's import to record this for reproducing the analysis
import refnx
print(refnx.version.version)

In [ ]:
# these are datasets used in refnx testing, distributed with every refnx install
pth = os.path.dirname(refnx.__file__)

# load the data
e361 = ReflectDataset(os.path.join(pth, 'analysis/test/', 'e361r.txt'))
e365 = ReflectDataset(os.path.join(pth, 'analysis/test/', 'e365r.txt'))
e366 = ReflectDataset(os.path.join(pth, 'analysis/test/', 'e366r.txt'))

In [ ]:
si = SLD(2.07, 'Si')
sio2 = SLD(3.47, 'SiO2')
polymer = SLD(2.0, 'polymer')
d2o = SLD(6.36, 'D2O')
h2o = SLD(-0.56, 'H2O')
cm3 = SLD(3.5, 'cm3.5')

sio2_l = sio2(30, 3)
sio2_l.thick.setp(vary=True, bounds=(1, 50))

# Each contrast uses the same polymer SLD. We account for contrast change
# using the volume fraction of solvent.
polymer_l = polymer(250, 3)
polymer_l.thick.setp(vary=True, bounds=(200, 300))
polymer_l.sld.real.setp(vary=True, bounds=(0.1, 2))
polymer_l.vfsolv.setp(vary=True, bounds=(0, 1))

# we're going to share the water/polymer roughness across all 3 datasets
water_poly_rough = Parameter(3, 'water_poly_rough')
d2o_l = d2o(0, water_poly_rough)
h2o_l = h2o(0, water_poly_rough)
cm3_l = cm3(0, water_poly_rough)

In [ ]:
structure361 = si | sio2_l | polymer_l | d2o_l
structure365 = si | sio2_l | polymer_l | cm3_l
structure366 = si | sio2_l | polymer_l | h2o_l

In [ ]:
model361 = ReflectModel(structure361)
model365 = ReflectModel(structure365)
model366 = ReflectModel(structure366)

model361.scale.setp(vary=True, bounds=(0.9, 1.1))
model361.bkg.setp(vary=True, bounds=(0.9e-8, 3e-5))
model365.scale.setp(vary=True, bounds=(0.9, 1.1))
model365.bkg.setp(vary=True, bounds=(0.9e-8, 3e-5))
model366.bkg.setp(vary=True, bounds=(0.9e-8, 3e-5))

In [ ]:
objective361 = Objective(model361, e361, transform=Transform('logY'))
objective365 = Objective(model365, e365, transform=Transform('logY'))
objective366 = Objective(model366, e366, transform=Transform('logY'))

global_objective = GlobalObjective([objective361, objective365, objective366])

In [ ]:
# create the fit instance
fitter = CurveFitter(global_objective)

res = fitter.fit('differential_evolution')

In [ ]:
global_objective.plot()
plt.legend()
plt.xlabel('Q')
plt.ylabel('logR');

In [ ]:
print(global_objective)

In [ ]:
fitter.sample(400, random_state=1)
fitter.sampler.reset()
res = fitter.sample(30, nthin=100, random_state=1)

In [ ]:
global_objective.corner();

In [ ]:
print(global_objective)